In [1]:
#  from pyecharts.globals import CurrentConfig, NotebookType
#  CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [1]:
import pandas as pd 
import re 
from pyecharts import options as opts 
from pyecharts.charts import PictorialBar,Line,Bar
from pyecharts.globals import SymbolType
from pyecharts.charts import Geo
from pyecharts.globals import ChartType
from pyecharts.commons.utils import JsCode

# 进行数据的读取

In [2]:
# 读取省份数据
# import os
# print(os.getcwd())
provincial = pd.read_csv(r'./provincial_capital')
china_city_code= pd.read_csv(r'./china-city-list.csv')
weather = pd.read_csv(r'./data/weather_data.csv')

In [3]:
# 查看天气总体情况
weather['wea'].value_counts()

# 转换天气变量
weather_dict = {
    "snow":100,
    "rain":80,
    "cloud":50,
    "overcast":60,
    "sun":20
}

In [4]:
# 提取省会城市的ID
provincial_data = pd.DataFrame()
for i in provincial['city'].values.tolist():
    for j in  china_city_code['City_CN'].values.tolist():
        if i == j :
            provincial_data = pd.concat([china_city_code[china_city_code['City_CN'] == j ],provincial_data])

# 按照城市分组
wea_group = weather.groupby('city').apply(lambda x : x[:])

# 提取出某一天的天气信息
zhongqiu = wea_group[wea_group['time'] == '周五（1日）']

# 提取出城市信息
zhongqiu_city = zhongqiu['city'].values.tolist()

# 获取温度信息
rege = r'(\d+)℃/(\d+)℃'

In [5]:
def trans_tem(tem):
    tmp_tea = re.match(rege,tem)
    mid_tem = (int(tmp_tea.group(1)) + int(tmp_tea.group(2))) / 2
    return mid_tem 

In [6]:
def check_weather(wea):
    if wea[-1:] == '晴':
        wea = weather_dict['sun']
    elif wea[-1:] == '云':
        wea = weather_dict['cloud']
    elif wea[-1:] == '雨':
        wea = weather_dict['rain']
    elif wea[-1:] == '阴':
        wea = weather_dict['overcast']
    return wea

# 获取降水和温度信息

In [7]:
# 进行数据的提取
weather_data = map(check_weather,zhongqiu['wea'].values.tolist())
weather_data = list(weather_data)

tem_data = map(trans_tem,zhongqiu['tem'].values.tolist())
tem_data = list(tem_data)
weather_data_list = list(weather_data)

# 降水和温度柱状图

In [8]:
pictorialbar = PictorialBar()
pictorialbar.add_xaxis(zhongqiu_city)
pictorialbar.add_yaxis("weather",
                       weather_data,
                       label_opts=opts.LabelOpts(is_show=False),
                       symbol_size=18,
                       symbol_repeat="fixed",
                       symbol_offset=[0, 0],
                       is_symbol_clip=True,
                       symbol=SymbolType.ROUND_RECT)

pictorialbar.add_yaxis("temperature",
                       tem_data,
                       label_opts=opts.LabelOpts(is_show=False),
                       symbol_size=18,
                       symbol_repeat="fixed",
                       symbol_offset=[0, 0],
                       is_symbol_clip=True,
                       symbol=SymbolType.ARROW)

pictorialbar.set_global_opts(
    title_opts=opts.TitleOpts(title="中秋节省会城市降雨和温度情况"),
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30),
                             is_show=True),
    yaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_show=False),
        axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(
            opacity=0)),
        axislabel_opts=opts.LabelOpts(formatter="{value}% 降水概率")),
    datazoom_opts=opts.DataZoomOpts())
pictorialbar.load_javascript()

In [9]:
pictorialbar.render_notebook()

In [10]:
line = Line()
line.add_xaxis(zhongqiu_city)
#line.add_yaxis("",tem_data)
#line.add_yaxis("",weather_data)
pictorialbar.overlap(line)
pictorialbar.render_notebook()

# 在双轴图中查看


In [ ]:
from pyecharts.render import make_snapshot
# from snapshot_phantomjs import snapshot
from snapshot_selenium import snapshot
def bar_chart() -> Bar:
    c = (
        Bar()
        .add_xaxis(zhongqiu_city)
        .add_yaxis("", weather_data)
        .extend_axis(yaxis=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} ℃"), interval=5))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="中秋节省会城市降雨和温度情况"),
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30),is_show=True),
            yaxis_opts=opts.AxisOpts(
            axistick_opts=opts.AxisTickOpts(is_show=False),
            axisline_opts=opts.AxisLineOpts(
            linestyle_opts = opts.LineStyleOpts(opacity=0)
            ),
            axislabel_opts = opts.LabelOpts(formatter="{value}% 降水概率")
        ),
            datazoom_opts=opts.DataZoomOpts()
        ) 
    )
    return c

In [19]:
bar = Bar()

In [21]:
bar.add_xaxis(zhongqiu_city)
bar.add_yaxis("", weather_data)
bar.extend_axis(yaxis=opts.AxisOpts(
    axislabel_opts=opts.LabelOpts(formatter="{value} ℃"), interval=5))
bar.set_global_opts(
        title_opts=opts.TitleOpts(title="中秋节省会城市降雨和温度情况"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-30),is_show=True),
        yaxis_opts=opts.AxisOpts(
            axistick_opts=opts.AxisTickOpts(is_show=False),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts = opts.LineStyleOpts(opacity=0)
            ),
            axislabel_opts = opts.LabelOpts(formatter="{value}% 降水概率")
        ),
        datazoom_opts=opts.DataZoomOpts()
)
bar.set_series_opts(itemstyle_opts={
    "normal":{
        "color":JsCode("""new echarts.graphic.LinearGradient(0,0,0,1,[{
            offset:0,
            color:'rgba(0,244,255,1)'
        },{
            offset:1,
            color:'rgba(0,77,167,1)'
        }],false)"""),
        "barBorderRadius":[30,30,30,30],
        "shadowColor":'rgb(0,160,221)',
    }
})
line = Line().add_xaxis(zhongqiu_city).add_yaxis("",tem_data,yaxis_index=1)
bar.overlap(line)
bar.render_notebook()

In [23]:
# make_snapshot(snapshot, bar_chart().render(), "./bar0.png")

# 几大城市天气情况

In [26]:
weather = pd.read_csv(r'./data/weather_data.csv')
beijing = weather[weather['city'] == '北京']
shanghai = weather[weather['city'] == '上海']
hangzhou = weather[weather['city'] == '杭州']
chengdu = weather[weather['city'] == '成都']

In [30]:
time_shanghai = shanghai['time'].values.tolist()
wea_shanghai = shanghai['wea'].values.tolist()
wea_list_shanghai = list(map(check_weather,wea_shanghai))
tem_shanghai = shanghai['tem'].values.tolist()
tem_list_shanghai = list(map(trans_tem,tem_shanghai))

In [33]:
line = Line()

In [43]:
line.add_xaxis(time_shanghai)
line.add_yaxis(
    "降水概率",
    wea_list_shanghai,
    is_smooth=True,
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]))
line.add_yaxis(
    "温度",
    tem_list_shanghai,
    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="min")]))
line.set_global_opts(
    title_opts=opts.TitleOpts(title = "上海中秋节前后降水和温度"),
    xaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts(rotate=-30),is_show=True)
)
lilinerender_notebook()